# Deep Convolutional Q-Learning for Pac-Man

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
# !pip install gymnasium
# !pip install "gymnasium[atari, accept-rom-license]"
# !apt-get install -y swig
# !pip install gymnasium[box2d]

### Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import IPython.display as ipythondisplay

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [3]:
class Network(nn.Module):
    def __init__(self, action_size, seed=42):
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, action_size)

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.bn1(self.conv1(state)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)

In [4]:
128 * 210 * 160 / 120

35840.0

## Part 2 - Training the AI

### Setting up the environment

In [5]:
import gymnasium as gym

env = gym.make("MsPacmanDeterministic-v0", full_action_space=False, render_mode="rgb_array")
state_shape = env.observation_space.shape
state_size = state_shape[0]
no_of_actions = env.action_space.n

print("State shape: ", state_shape)
print("State Size: ", state_size)
print("Number of actions: ", no_of_actions)

State shape:  (210, 160, 3)
State Size:  210
Number of actions:  9


/Users/mufaddalhamid/Documents/Learn/AI_Course/.conda/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


### Initializing the hyperparameters

In [6]:
learning_rate = 5e-4
minibatch_size = 64
gamma = 0.99  # discount factor

### Preprocessing the frames

In [7]:
from PIL import Image
from torchvision import transforms


def preprocess_frame(frame):
    frame = Image.fromarray(frame)
    preprocess = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])
    
    return preprocess(frame).unsqueeze(0)



### Implementing the DCQN class

In [8]:
class Agent:
    def __init__(self, action_size):
        self.device = torch.device("mps")
        self.action_size = action_size
        self.local_qnetwork = Network(action_size).to(self.device)
        self.target_qnetwork = Network(action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)
        self.memory = deque(maxlen=10000)

    def step(self, state, action, reward, next_state, done) -> None:
        state = preprocess_frame(state)
        next_state = preprocess_frame(next_state)
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > minibatch_size:
            experiences = random.sample(self.memory, minibatch_size)
            self.learn(experiences, gamma)

    def act(self, state, eps=0.0) -> int:
        """Returns actions for given state as per current policy."""
        state = preprocess_frame(state).to(self.device)
        self.local_qnetwork.eval()
        with torch.no_grad():
            action_values = self.local_qnetwork(state)
        self.local_qnetwork.train()
        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, discount_factor):
        states, actions, rewards, next_states, dones = zip(*experiences)
        states = torch.from_numpy(np.vstack(states)).float().to(self.device)
        actions = torch.from_numpy(np.vstack(actions)).long().to(self.device)
        rewards = torch.from_numpy(np.vstack(rewards)).float().to(self.device)
        next_states = torch.from_numpy(np.vstack(next_states)).float().to(self.device)
        dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(self.device)
        next_q_targets = (
            self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
        )
        q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
        q_expected = self.local_qnetwork(states).gather(1, actions)
        loss = F.mse_loss(q_expected, q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

### Initializing the DCQN agent

In [9]:
agent = Agent(no_of_actions)
# agent.local_qnetwork.load_state_dict(torch.load("checkpoint.pth"))

### Training the DCQN agent

In [10]:
import imageio

no_of_episodes = 2000
max_timesteps = 10000  # max number of timesteps in an episode
epsilon_start = 1
epsilon_end = 0.01
epsilon_decay = 0.995
eps = epsilon_start
scores = deque(maxlen=100)  # list containing scores from last 100 episodes

In [13]:
os.environ['IMAGEIO_FFMPEG_EXE'] = "/opt/homebrew/bin/ffmpeg"

for episode in range(1, no_of_episodes + 1):
    state, _ = env.reset()
    score = 0
    frames = []
    for t in range(max_timesteps):
        action = agent.act(state, eps)
        next_state, reward, done, _, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        
        screen = env.render()
        frames.append(screen)
        if done:
            break
    imageio.mimsave(f'videos/pacman/video_{episode}_{score:.0f}.mp4', frames, fps=30, macro_block_size=1)
    torch.save(agent.local_qnetwork.state_dict(), f"checkpoints/pacman/checkpoint_{episode}_{score:.0f}.pth")
    scores.append(score)
    eps = max(epsilon_end, epsilon_decay * eps)
    print(f"\rEpisode {episode}\tAverage Score: {np.mean(scores):.2f}\tCurrent Score: {score:.2f}    ", end="")
    if episode % 100 == 0:
        print(f"\rEpisode {episode}\tAverage Score: {np.mean(scores):.2f}                          ")
    if np.mean(scores) >= 500:
        print(f"\nEnvironment solved in {episode:d} episodes!\tAverage Score: {np.mean(scores):.2f}")
        break

Episode 2	Average Score: 229.13	Current Score: 170.00    

KeyboardInterrupt: 

## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

os.environ['IMAGEIO_FFMPEG_EXE'] = "/opt/homebrew/bin/ffmpeg"

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'MsPacmanDeterministic-v0')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

/Users/mufaddalhamid/Documents/Learn/AI_Course/.conda/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment MsPacmanDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


In [ ]:
import os
os.system("say 'your program has finished'")

0